In [ ]:
!pip install -qq --upgrade pip
!pip install -qq --upgrade transformers accelerate bitsandbytes peft datasets
!pip install oxenai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 MB 8.3 MB/s  0:00:05
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 54.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 81.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 783.6/783.6 kB 43.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 MB 42.9 MB/s  0:00:00
  Attempting uninstall: polars
    Found existing installation: polars 1.31.0
    Uninstalling polars-1.31.0:
      Successfully uninstalled polars-1.31.0
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6/6 [oxenai]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
googl

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TextStreamer
from oxen import RemoteRepo
import oxen
from peft import PeftModel, PeftConfig

In [ ]:
from google.colab import userdata
oxen_key = userdata.get('oxen_key') # OxenAI API key
huggingface_key = userdata.get('huggingface_key') # Huggingface API key

from oxen.auth import config_auth
config_auth(oxen_key)

from huggingface_hub import login
login(token=huggingface_key, new_session=False)

# Choose the type of finetuned model you want to load

## Load full finetuned model

In [ ]:
def load_fine_tuned(model_name, version=None):
    print(f"Loading fine-tuned model from Hugging Face: {model_name}, revision={version}")

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        revision=version,
        device_map="auto",
        torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
        use_safetensors=True
    )

    tokenizer = AutoTokenizer.from_pretrained(
        model_name,
        use_fast=True
    )

    return model, tokenizer

## Load base model

In [ ]:
def load_base_model(model_name):
    print(f"Loading base model: {model_name}")
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype="auto",
        device_map="auto"
    )
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    return model, tokenizer

## Load model

In [ ]:
base_model="Qwen/Qwen3-0.6B"
dataset_type = "SingleTable" # Don't change this value
model_name_single_table = "ZiDuck/Qwen3-0.6B-Text2SQL"
model_name_bird = "ZiDuck/SFT-Qwen3-0.6B-Text2SQL-MiniBIRD"
mode = "base" # peft | fine_tune | base


model_name = model_name_single_table if dataset_type == "SingleTable" else model_name_bird
if "fine_tune" == mode:
    model, tokenizer = load_fine_tuned(model_name)
elif "peft" == mode:
    # model, tokenizer = load_peft_model(model_repo_name, version, base_model, checkpoint=checkpoint)
    print("not_support")
elif "base" == mode:
    model, tokenizer = load_base_model(base_model)
    print("not_support")
else:
    print(f"Unknown mode: {mode}")

model

Loading base model: Qwen/Qwen3-0.6B
not_support


Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 1024)
    (layers): ModuleList(
      (0-27): 28 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear(in_features=1024, out_features=2048, bias=False)
          (k_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=1024, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (up_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (down_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): Qwen3RMSNorm((1024,), eps=1e-06)
        (post_attention_layer

# Evaluation

In [ ]:
import re

def remove_think_block(text: str) -> str:
    text = re.sub(r"<think>.*?</think>", "", text, flags=re.DOTALL)
    return text.strip()

In [ ]:
# For Single Table Dataset
def predictSingleTable(tokenizer: AutoTokenizer, model: AutoModelForCausalLM, sample: dict):
    system_prompt = """Write a SQL statement that is equivalent to the natural language user query below. You are given the schema in the format of a CREATE TABLE SQL statement. Assume the table is called "df". DO NOT give any preamble or extra characters or markdown just the SQL query in plain text. Make sure the SQL query is on one line."""
    user_prompt = f"Schema:\n{sample['schema']}\n\nUser Query:\n{sample['query']}\n\nSQL Query:\n"

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    streamer = TextStreamer(tokenizer)
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=1024,
        # streamer=streamer
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    return text, response

In [ ]:
# For BIRD minidev Dataset
def predictBIRD(tokenizer: AutoTokenizer, model: AutoModelForCausalLM, sample: dict):
    system_prompt = """Task Overview:
You are a data science expert. Below, you are provided with a database schema and a natural language question. Your task is to understand the schema and generate a valid SQL query to answer the question.

Database Engine:
SQLite"""

    user_prompt = f"""Database Schema:
{sample['schema']}
This schema describes the database's structure, including tables, columns, primary keys, foreign keys, and any relevant relationships or constraints.

Question:
{sample['query']}

Instructions:
- Make sure you only output the information that is asked in the question. If the question asks for a specific column, make sure to only include that column in the SELECT clause, nothing more.
- Do NOT hallucinate: only use tables, columns, and values that exist in the provided schema.
- The generated query should return all of the information asked in the question without any missing or extra information.
- Before generating the final SQL query, please think through the steps of how to write the query.
- Keep the SQL minimal: no extra joins, filters, grouping, ordering, or aliases unless required.
- DO NOT give any preamble or extra characters or markdown, just the SQL query in plain text on a single line with no line breaks or indentation. DO NOT use any code fences or the substring ```sql in the output.
- Take a deep breath and think step by step to find the correct SQL query.

SQL Query:"""

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    streamer = TextStreamer(tokenizer)
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=1024,
        # streamer=streamer
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    raw_response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    response = remove_think_block(raw_response)

    return text, response

In [ ]:
import pandas as pd

def evaluate(
    model,
    tokenizer,
    version,
    repo_name,
    repo_branch,
    repo_file,
    predict,
    data_set_type = "SingleTable",
    save_every=10
):
    data_repo = RemoteRepo(repo_name)
    data_repo.checkout(repo_branch)
    data_repo.download(repo_file)

    out_file = f"{version}.parquet"
    full_path = os.path.join("results", out_file)
    os.makedirs(os.path.dirname(full_path), exist_ok=True)
    print(f"Saving results to {full_path}")

    if data_repo.file_exists(full_path):
        print(f"Downloading {full_path}")
        data_repo.download(full_path, revision=repo_branch)
        df = pd.read_parquet(full_path)
    else:
        df = pd.read_parquet(repo_file)
        # Initialize the prediction column with None values
        df['prediction'] = None
        df['raw_prompt'] = None

    # Track when we last committed
    last_commit = 0

    for idx, row in df.iterrows():
        data = {
            'query': row['full_question'] if data_set_type == "BIRD" else row['query'],
            'schema': row['schema']
        }
        print(f"Row {idx}: {data}")

        prediction = df.at[idx, 'prediction']
        if prediction is not None and prediction != "":
            print(f"Already got prediction for idx {idx}")
            continue

        # Make prediction and store directly in the dataframe
        raw_prompt, prediction = predict(tokenizer, model, data)
        print(f"Prediction: {prediction}")
        ground_truth = row['SQL'] if data_set_type == "BIRD" else row['sql']
        print(f"Ground Tru: {ground_truth}")
        print("="*80)
        df.at[idx, 'prediction'] = prediction
        df.at[idx, 'raw_prompt'] = raw_prompt

        # Commit every N rows
        if (idx + 1) % save_every == 0:
            print(f"Saving checkpoint at row {idx + 1}")
            df.to_parquet(full_path)
            data_repo.add(full_path, dst="results")
            data_repo.commit(f"Adding {idx} results for {version}")
            last_commit = idx + 1

    # Final save for any remaining rows
    if len(df) > last_commit:
        print(f"Final save with {len(df) - last_commit} additional rows")
        df.to_parquet(full_path)
        data_repo.add(full_path, dst="results")
        data_repo.commit(f"Adding all results for {version}")

    print("Prediction process complete!")

In [ ]:
version = "Qwen3-0.6B-Single-Table-Predictions"
data_repo_name = "ZiDuck/text2sql" # Repository name on OxenAI
data_repo_branch = "main"
data_repo_file = "dataset/parquet_format/single-table_dev.parquet"
save_every = 10


# Run the full evaluation
evaluate(
    model,
    tokenizer,
    version if version != "" else base_model,
    data_repo_name,
    data_repo_branch,
    data_repo_file,
    predictBIRD if dataset_type == "BIRD" else predictSingleTable,
    dataset_type,
    save_every=save_every
)

Saving results to results/Qwen3-0.6B-Single-Table-Predictions.parquet
Row 0: {'query': 'In season is 2008 09, how many wins did they have?', 'schema': 'CREATE TABLE df ("Season" text, "GP" real, "W (OT/SO)" text, "L (OT/SO)" text, "Pts" real, "Pts/GP" text, "GF – GA" text, "Rank (league/conference)" text, "Top Scorer" text)'}
Prediction: SELECT COUNT(*) FROM df WHERE "Season" = '2008 09';
Ground Tru: SELECT "W (OT/SO)" FROM df WHERE "Season" = '2008–09'
Row 1: {'query': 'find me all the rows where category != prediction', 'schema': 'CREATE TABLE df (text VARCHAR, category VARCHAR, prediction VARCHAR);'}
Prediction: CREATE TABLE df (text VARCHAR, category VARCHAR, prediction VARCHAR);  
SELECT * FROM df WHERE category != prediction;
Ground Tru: 
SELECT * FROM df WHERE category != prediction;

Row 2: {'query': 'What are the investors of entrepreneurs and the corresponding number of entrepreneurs invested by each investor. Visualize by pie chart.', 'schema': 'CREATE TABLE df (Entrepreneur

# Test the finetune model

In [ ]:
def test_prompt(tokenizer, model, text: str, max_new_tokens: int = 256):
    try:
        # Encode
        try:
            inputs = tokenizer([text], return_tensors="pt").to(model.device)
        except Exception as e:
            print(f"[Tokenizer Error] {e}")
            return None

        # Generate
        try:
            outputs = model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                do_sample=False
            )
        except Exception as e:
            print(f"[Model Generation Error] {e}")
            return None

        # Decode
        try:
            generated_ids = outputs[0, inputs["input_ids"].shape[1]:]
            response = tokenizer.decode(generated_ids, skip_special_tokens=True)
        except Exception as e:
            print(f"[Decoding Error] {e}")
            return None

        return response

    except Exception as e:
        # Catch-all (phòng trường hợp lỗi chưa lường trước)
        print(f"[Unexpected Error] {e}")
        return None

## Single table dataset

In [ ]:
question = "In season is 2008 09, how many wins did they have?"
schema = "CREATE TABLE df (`Season` TEXT, `GP` REAL, `W (OT/SO)` TEXT, `L (OT/SO)` TEXT, `Pts` REAL, `Pts/GP` TEXT, `GF – GA` TEXT, `Rank (league/conference)` TEXT, `Top Scorer` TEXT)"
gold_sql = "SELECT `W (OT/SO)` FROM df WHERE `Season` = '2008–09'"

In [ ]:
data_test = {
    'query': question,
    'schema': schema
}

# Make prediction and store directly in the dataframe
raw_prompt, prediction = predictSingleTable(tokenizer, model, data_test)

print(f"Prediction: {prediction}")
print(f"Ground Tru: {gold_sql}")
print(f"Raw prompt: {raw_prompt}")

Prediction: SELECT "W (OT/SO)" FROM df WHERE "Season" = '2008–09'
Ground Tru: SELECT `W (OT/SO)` FROM df WHERE `Season` = '2008–09'
Raw prompt: <|im_start|>system
Write a SQL statement that is equivalent to the natural language user query below. You are given the schema in the format of a CREATE TABLE SQL statement. Assume the table is called "df". DO NOT give any preamble or extra characters or markdown just the SQL query in plain text. Make sure the SQL query is on one line.<|im_end|>
<|im_start|>user
Schema:
CREATE TABLE df (`Season` TEXT, `GP` REAL, `W (OT/SO)` TEXT, `L (OT/SO)` TEXT, `Pts` REAL, `Pts/GP` TEXT, `GF – GA` TEXT, `Rank (league/conference)` TEXT, `Top Scorer` TEXT)

User Query:
In season is 2008 09, how many wins did they have?

SQL Query:
<|im_end|>
<|im_start|>assistant
<think>

</think>




## BIRD minidev dataset

In [ ]:
question = "ratio of customers who pay in EUR against customers who pay in CZK = count(Currency = 'EUR') / count(Currency = 'CZK').What is the ratio of customers who pay in EUR against customers who pay in CZK?"
schema = """
CREATE TABLE customers (
    CustomerID integer, -- example: [3, 5]
    Segment text, -- client segment, example: ['SME', 'LAM']
    Currency text, -- example: ['CZK', 'EUR']
    PRIMARY KEY (CustomerID)
);

CREATE TABLE gasstations (
    GasStationID integer, -- example: [44, 45]
    ChainID integer, -- example: [13, 6]
    Country text, -- example: ['CZE', 'SVK']
    Segment text, -- chain segment, example: ['Value for money', 'Premium']
    PRIMARY KEY (GasStationID)
);

CREATE TABLE products (
    ProductID integer, -- example: [1, 2]
    Description text, -- example: ['Rucní zadání', 'Nafta']
    PRIMARY KEY (ProductID)
);

CREATE TABLE transactions_1k (
    TransactionID integer, -- example: [1, 2]
    `Date` date, -- example: ['2012-08-24', '2012-08-23']
    `Time` text, -- example: ['09:41:00', '10:03:00']
    CustomerID integer, -- example: [31543, 46707]
    CardID integer, -- example: [486621, 550134]
    GasStationID integer, -- example: [3704, 656]
    ProductID integer, -- example: [2, 23]
    Amount integer, -- example: [28, 18]
    Price real, -- example: [672.64, 430.72]
    PRIMARY KEY (TransactionID)
);

CREATE TABLE yearmonth (
    CustomerID integer, -- example: [39, 63]
    `Date` text, -- example: ['201112', '201201']
    Consumption real, -- example: [528.3, 1598.28]
    PRIMARY KEY (CustomerID, `Date`),
    CONSTRAINT fk_yearmonth_customerid FOREIGN KEY (CustomerID) REFERENCES customers (CustomerID)
);
"""
gold_sql = "SELECT CAST(SUM(IIF(Currency = 'EUR', 1, 0)) AS FLOAT) / SUM(IIF(Currency = 'CZK', 1, 0)) AS ratio FROM customers"

In [ ]:
data_test = {
    'query': question,
    'schema': schema
}

# Make prediction and store directly in the dataframe
raw_prompt, prediction = predictBIRD(tokenizer, model, data_test)

print(f"Prediction: {prediction}")
print(f"Ground Tru: {gold_sql}")
print(f"Raw prompt: {raw_prompt}")

Prediction: SELECT CAST(SUM(CASE WHEN T1.Currency = 'CZK' THEN 1 ELSE 0 END) AS REAL) / SUM(CASE WHEN T1.Currency = 'EUROPE' THEN 1 ELSE 0 END) FROM customers AS T1 INNER JOIN yearmonth AS T2 ON T1.CustomerID = T2.CustomerID
Ground Tru: SELECT CAST(SUM(IIF(Currency = 'EUR', 1, 0)) AS FLOAT) / SUM(IIF(Currency = 'CZK', 1, 0)) AS ratio FROM customers
Raw prompt: <|im_start|>system
Task Overview:
You are a data science expert. Below, you are provided with a database schema and a natural language question. Your task is to understand the schema and generate a valid SQL query to answer the question.

Database Engine:
SQLite<|im_end|>
<|im_start|>user
Database Schema:

CREATE TABLE customers (
    CustomerID integer, -- example: [3, 5]
    Segment text, -- client segment, example: ['SME', 'LAM']
    Currency text, -- example: ['CZK', 'EUR']
    PRIMARY KEY (CustomerID)
);

CREATE TABLE gasstations (
    GasStationID integer, -- example: [44, 45]
    ChainID integer, -- example: [13, 6]
    Co